# Summary
Read the class info file for MiBIG entries, and report probability tables.

In [6]:
import os
import urllib
import glob
import json
import time

In [7]:
mibig_folder = "/mnt/scratch/louwe015/mibig_json_2.0/"

base_path = "/mnt/scratch/louwe015/NPLinker/classifying/mibig_classifications/"
out_base = os.path.split(base_path[:-1])[0]
mibig_classes = os.path.join(out_base, "All_MIBiG_compounds_with_AS_BGC_CF_NPC_classes.txt")
print("mibig_classes exists is", os.path.isfile(mibig_classes))

mibig_classes exists is True


## Reading file
Making dict of {chem_id: [[bgc_classes], [chem_classes]]}, where bgc_classes is [class, subclass, AS_class], and chem_classes is the CF/NPC classes in order of input file

In [29]:
classes_dict = {}
with open(mibig_classes) as inf:
    header = inf.readline()
    print(header)
    for line in inf:
        elems = line.strip().split("\t")
        chem_id = elems.pop(0)
        class_base = elems.pop(0).split(',')
        classes = [cls.partition(':')[0] for cls in class_base]
        sub_classes = [cls for cls in class_base if cls.split(":")[1]]
        as_classes = elems.pop(0).split(',')

        bgc_classes = [classes, sub_classes, as_classes]        
        chem_classes = [chem_cls.split('; ') for chem_cls in elems[2:]]
        classes_dict[chem_id] = [bgc_classes, chem_classes]

print(classes_dict[chem_id])  # example
print(classes_dict['BGC0000020_maytansine'])
classes_dict['BGC0000001_abyssomicin C']

compound_name	class:subclass	AS_classes	smiles	inchi_key	kingdom	superclass	class	subclass	direct_parent	class_results	superclass_results	pathway_results	isglycoside

[[['Other'], [], ['phosphonate']], [[''], [''], [''], [''], [''], [''], [''], [''], ['0']]]
[[['Polyketide'], [], ['T1PKS', 'NRPS-like']], [['Organic compounds'], ['Phenylpropanoids and polyketides'], ['Macrolactams'], [''], ['Macrolactams'], ['Ansa macrolides'], ['Macrolides'], ['Polyketides'], ['0']]]


[[['Polyketide'], ['Polyketide:Other'], ['T1PKS']],
 [['Organic compounds'],
  ['Organoheterocyclic compounds'],
  ['Oxanes'],
  [''],
  ['Oxanes'],
  ['Spirotetronate macrolides'],
  ['Macrolides'],
  ['Polyketides'],
  ['0']]]

In [30]:
# creating legend from the header
s_h = header.strip().split('\t')
legend = [['BGC_class']+s_h[1:3], s_h[5:]]
print(legend)

[['BGC_class', 'class:subclass', 'AS_classes'], ['kingdom', 'superclass', 'class', 'subclass', 'direct_parent', 'class_results', 'superclass_results', 'pathway_results', 'isglycoside']]


## Do comparisons between bgc and chem classes